In [82]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

! pip install geopy
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [83]:
# Read data from file 'filename.csv' 
# (in the same directory that your python process is based)
# Control delimiters, rows, column names with read_csv (see later) 
data = pd.read_csv("distritos_saopaulo.csv") 
# Preview the first 5 lines of the loaded data 
data.head()

,Regions,Boroughs,Latitude,Longitude
0,Centro,Bela Vista,-23.552128,-46.635161
1,Centro,Bom Retiro,-23.525656,-46.651618
2,Centro,Cambuci,-23.565619,-46.630160
3,Centro,Consolacao,-23.554083,-46.655463
4,Centro,Liberdade,-23.565311,-46.641115


In [84]:
# instantiate the dataframe
neighborhoods = data

In [85]:
print('The dataframe has {} Regions and {} Boroughs.'.format(
        len(neighborhoods['Regions'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 Regions and 97 Boroughs.


In [86]:
address = 'Sao Paulo City, SP'

geolocator = Nominatim(user_agent="sp_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Sao Paulo City are {}, {}.'.format(latitude, longitude))

The geographical coordinate of Sao Paulo City are -23.5506507, -46.6333824.


In [87]:
# create map of New York using latitude and longitude values
map_saopaulo = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, boroughs, regions in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Regions'], neighborhoods['Boroughs']):
    label = '{}, {}'.format(regions, boroughs)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=1,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_saopaulo)  
    
map_saopaulo

In [88]:
downtown_data = neighborhoods[neighborhoods['Regions'] == 'Oeste'].reset_index(drop=True)
downtown_data.head()

,Regions,Boroughs,Latitude,Longitude
0,Oeste,Butanta,-23.571924,-46.709588
1,Oeste,Morumbi,-23.600257,-46.725546
2,Oeste,Raposo Tavares,-23.591375,-46.772341
3,Oeste,Rio Pequeno,-23.566599,-46.758615
4,Oeste,Vila Sonia,-23.596152,-46.736559


In [89]:
address = 'Sao Paulo, SP'

geolocator = Nominatim(user_agent="sp_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Centro are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Centro are -23.5506507, -46.6333824.


In [90]:
# create map of New York using latitude and longitude values
map_downtown = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, boroughs, regions in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Regions'],neighborhoods['Boroughs']):
    label = '{}, {}'.format(regions, boroughs)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=6,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_downtown)  
    
map_downtown

In [91]:
CLIENT_ID = 'WNET5ZNSB0CHPI4CUCTQV0PHFY0EP00LDT4MIVFK13ZEMEZ0' # your Foursquare ID
CLIENT_SECRET = 'WP3UWH5KQG0XH43LGDPH4ZUV34KD5PQJMYLGQLFVO2FURFMM' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: WNET5ZNSB0CHPI4CUCTQV0PHFY0EP00LDT4MIVFK13ZEMEZ0
CLIENT_SECRET:WP3UWH5KQG0XH43LGDPH4ZUV34KD5PQJMYLGQLFVO2FURFMM


In [92]:
neighborhood_latitude = neighborhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = neighborhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = neighborhoods.loc[0, 'Boroughs'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Bela Vista are -23.5521281, -46.6351612.


In [93]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=WNET5ZNSB0CHPI4CUCTQV0PHFY0EP00LDT4MIVFK13ZEMEZ0&client_secret=WP3UWH5KQG0XH43LGDPH4ZUV34KD5PQJMYLGQLFVO2FURFMM&v=20180605&ll=-23.5521281,-46.6351612&radius=500&limit=100'

In [94]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5faef20812d8516a986a1524'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Sé',
  'headerFullLocation': 'Sé, São Paulo',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 59,
  'suggestedBounds': {'ne': {'lat': -23.547628095499995,
    'lng': -46.63026143168324},
   'sw': {'lat': -23.556628104500007, 'lng': -46.64006096831676}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5477b866498edd0ef825a142',
       'name': 'Izakayada',
       'location': {'address': 'Pç. Carlos Gomes, 61A',
        'lat': -23.55315679592638,
        'lng': -46.63563945035646,
        'labeledLatLngs': [{'label': 'display',
          'lat': -23.55315679

In [95]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [96]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Izakayada,Sake Bar,-23.553157,-46.635639
1,Kopenhagen,Chocolate Shop,-23.551759,-46.635370
2,Restaurante Mugui,Japanese Restaurant,-23.554111,-46.634212
3,Sushi Isao,Sushi Restaurant,-23.554134,-46.634245
4,Cine Joia,Nightclub,-23.553354,-46.635767


In [97]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

59 venues were returned by Foursquare.


In [98]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [99]:
# type your answer here
saopaulo_venues = getNearbyVenues(names=neighborhoods['Boroughs'],
                                   latitudes=neighborhoods['Latitude'],
                                   longitudes=neighborhoods['Longitude']
                                  )

Bela Vista
Bom Retiro
Cambuci
Consolacao
Liberdade
Republica
Santa Cecília
Se
Aricanduva
Carrao
Vila Formosa
Cidade Tiradentes
Ermelino Matarazzo
Ponte Rasa
Guaianases
Lajeado
Itaim Paulista
Vila Curuca
Itaquera
Cidade Lider
Jose Bonifacio
Parque do Carmo
Mooca Agua Rasa
Belem
Bras
Moóca
Pari
Tatuape
Penha
Artur Alvim
Cangaíba
Vila Matilde
Sao Mateus
Sao Rafael
Sao Miguel
Jardim Helena
Vila Jacui
Sapopemba
Vila Prudente
Sao Lucas
Casa Verde
Cachoeirinha
Limao
Brasilandia
Freguesia do O


KeyError: 'groups'

In [ ]:
print(saopaulo_venues.shape)
saopaulo_venues.head()

In [ ]:
saopaulo_venues.groupby('Neighborhood').count()

In [ ]:
print('There are {} uniques categories.'.format(len(saopaulo_venues['Venue Category'].unique())))

In [ ]:
# one hot encoding
saopaulo_onehot = pd.get_dummies(saopaulo_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
saopaulo_onehot['Neighborhood'] = saopaulo_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [saopaulo_onehot.columns[-1]] + list(saopaulo_onehot.columns[:-1])
saopaulo_onehot= saopaulo_onehot[fixed_columns]

saopaulo_onehot.head()

In [ ]:
saopaulo_onehot.shape

In [ ]:
saopaulo_venues_categories= saopaulo_venues.drop(['Neighborhood','Neighborhood Latitude', 'Neighborhood Longitude', 'Venue Latitude', 'Venue Longitude', 'Venue'], axis=1)
saopaulo_venues_categories2 = saopaulo_venues_categories.groupby('Venue Category').size().reset_index(name='Counts')

# sort Brand - descending order
saopaulo_venues_categories2.sort_values(by=['Counts'], inplace=True, ascending=False)

saopaulo_venues_categories2

In [ ]:
# split datframe into 2 parts
#first comprising the venues categories with more than 10
#saopaulo_venues_categories2['Grouped Venue Categories'] = np.where(saopaulo_venues_categories2['Counts']>=10, saopaulo_venues_categories2['Venue Category'] , 'Grouped Venue Categories')
#saopaulo_venues_categories2['Counts2'] = np.where(saopaulo_venues_categories2['Counts']>=10, saopaulo_venues_categories2['Counts'] ,  saopaulo_venues_categories2['Counts'] )

#saopaulo_venues_categories2 

#saopaulo_venues_categories3= saopaulo_venues_categories2.drop(['Venue Category','Counts2'], axis=1)

#saopaulo_venues_categories3 = saopaulo_venues_categories2.groupby('Grouped Venue Categories').size().reset_index(name='Counts2')

#saopaulo_venues_categories3


In [ ]:
#! pip install squarify
#import squarify 

#labels = saopaulo_venues_categories3.apply(lambda x: str(x[0]) + "\n (" + str(x[1]) + ")", axis=1)
#sizes = saopaulo_venues_categories3['Counts'].values.tolist()
#colors = [plt.cm.Spectral(i/float(len(labels))) for i in range(len(labels))]

# Draw Plot
#plt.figure(figsize=(12,8), dpi= 80)
#squarify.plot(sizes=sizes, label=labels, color=colors, alpha=.8)

# Decorate
#plt.title('Treemap of Vechile Class')
#plt.axis('off')
#plt.show()

In [100]:
saopaulo_grouped = saopaulo_onehot.groupby('Neighborhood').mean().reset_index()
saopaulo_grouped

,Neighborhood,Acai House,Accessories Store,Adult Boutique,Airport Service,Airport Terminal,American Restaurant,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Baiano Restaurant,Bakery,Bank,Bar,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Bookstore,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Buffet,Building,Burger Joint,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Candy Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Cafeteria,College Quad,College Theater,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Dive Bar,Dog Run,Donburi Restaurant,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store,Empada House,Empanada Restaurant,Escape Room,Event Service,Fabric Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Festival,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Go Kart Track,Gourmet Shop,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,Hobby Shop,Home Service,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Lighthouse,Lighting Store,Lingerie Store,Liquor Store,Lottery Retailer,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mineiro Restaurant,Miscellaneous Shop,Mobile Phone Shop,Molecular Gastronomy Restaurant,Motel,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Newsstand,Nightclub,Non-Profit,Northeastern Brazilian Restaurant,Office,Optical Shop,Organic Grocery,Other Nightlife,Outdoor Supply Store,Outdoors & Recreation,Outlet Store,Paintball Field,Paper / Office Supplies Store,Park,Pastelaria,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Recording Studio,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Rest Area,Restaurant,Road,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Southeastern Brazilian Restaurant,Southern Brazilian Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Club,Stadium,Steakhouse,Street Art,Street Food Gathering,Student Center,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tapiocaria,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Tour Provider,Toy / Game Store,Train Station,Travel Agency,Tunnel,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Water Park,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Alto de Pinheiros,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0

In [101]:
saopaulo_grouped.shape

(91, 275)

In [102]:
num_top_venues = 10

for hood in saopaulo_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp =  saopaulo_grouped[ saopaulo_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Alto de Pinheiros----
                           venue  freq
0           Brazilian Restaurant  0.09
1                            Bar  0.07
2                         Bakery  0.03
3                          Plaza  0.03
4                    Pizza Place  0.03
5                     Restaurant  0.03
6                       Pharmacy  0.03
7            Martial Arts School  0.03
8  Paper / Office Supplies Store  0.03
9                 Gymnastics Gym  0.02


----Anhanguera----
                   venue  freq
0           Soccer Field   0.5
1           Tennis Court   0.5
2                 Office   0.0
3  Outdoors & Recreation   0.0
4   Outdoor Supply Store   0.0
5        Other Nightlife   0.0
6        Organic Grocery   0.0
7           Optical Shop   0.0
8             Acai House   0.0
9        Paintball Field   0.0


----Aricanduva----
                 venue  freq
0               Bakery  0.21
1             Pharmacy  0.14
2  Sporting Goods Shop  0.07
3       Gymnastics Gym  0.07
4           Paste

In [103]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [104]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = saopaulo_grouped['Neighborhood']

for ind in np.arange(saopaulo_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(saopaulo_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(15)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Alto de Pinheiros,Brazilian Restaurant,Bar,Pizza Place,Restaurant,Paper / Office Supplies Store
1,Anhanguera,Tennis Court,Soccer Field,Yoga Studio,Food,Farmers Market
2,Aricanduva,Bakery,Pharmacy,Pastelaria,Café,Gym
3,Artur Alvim,Pizza Place,Soccer Stadium,Grocery Store,Burger Joint,Clothing Store
4,Barra Funda,Nightclub,Bar,Brazilian Restaurant,Café,Sandwich Place
5,Bela Vista,Cosmetics Shop,Japanese Restaurant,Grocery Store,Bakery,Bookstore
6,Belem,Diner,Kids Store,Brewery,Fried Chicken Joint,Brazilian Restaurant
7,Bom Retiro,Café,Pizza Place,Dive Bar,Brazilian Restaurant,Pub
8,Bras,Shopping Mall,Clothing Store,Farmers Market,Hotel,Food & Drink Shop
9,Brasilandia,BBQ Joint,Pizza Place,Diner,Pharmacy,Recording Studio


In [105]:
# set number of clusters
kclusters = 10

saopaulo_grouped_clustering = saopaulo_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(saopaulo_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 2, 8, 6, 1, 1, 1, 1, 1, 6], dtype=int32)

In [106]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

saopaulo_merged = neighborhoods

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
saopaulo_merged = saopaulo_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Boroughs')

# drop rows when cluster is NaN
saopaulo_merged.dropna(how='any', inplace=True)
#saopaulo_merged. = df.reset_index(drop=True)

# converting 'Weight' from float to int 
saopaulo_merged['Cluster Labels'] = saopaulo_merged['Cluster Labels'].astype(int) 

saopaulo_merged.head() # check the last columns!

,Regions,Boroughs,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Centro,Bela Vista,-23.552128,-46.635161,1,Cosmetics Shop,Japanese Restaurant,Grocery Store,Bakery,Bookstore
1,Centro,Bom Retiro,-23.525656,-46.651618,1,Café,Pizza Place,Dive Bar,Brazilian Restaurant,Pub
2,Centro,Cambuci,-23.565619,-46.630160,1,Pizza Place,Pharmacy,Pet Store,Farmers Market,Grocery Store
3,Centro,Consolacao,-23.554083,-46.655463,1,Café,Bar,Brazilian Restaurant,Dance Studio,Theater
4,Centro,Liberdade,-23.565311,-46.641115,1,Bakery,Brazilian Restaurant,Diner,Pizza Place,Coffee Shop


In [107]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.tab10(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(saopaulo_merged['Latitude'], saopaulo_merged['Longitude'], saopaulo_merged['Boroughs'], saopaulo_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster -1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
    
      
map_clusters

In [108]:
saopaulo_merged.loc[saopaulo_merged['Cluster Labels'] == 0, saopaulo_merged.columns[[1] + list(range(5, saopaulo_merged.shape[1]))]]

,Boroughs,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
41,Cachoeirinha,Farmers Market,Grocery Store,Yoga Studio,Food,Fast Food Restaurant


In [109]:
saopaulo_merged.loc[saopaulo_merged['Cluster Labels'] ==1 , saopaulo_merged.columns[[1] + list(range(5, saopaulo_merged.shape[1]))]]

,Boroughs,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Bela Vista,Cosmetics Shop,Japanese Restaurant,Grocery Store,Bakery,Bookstore
1,Bom Retiro,Café,Pizza Place,Dive Bar,Brazilian Restaurant,Pub
2,Cambuci,Pizza Place,Pharmacy,Pet Store,Farmers Market,Grocery Store
3,Consolacao,Café,Bar,Brazilian Restaurant,Dance Studio,Theater
4,Liberdade,Bakery,Brazilian Restaurant,Diner,Pizza Place,Coffee Shop
5,Republica,Brazilian Restaurant,Coffee Shop,Italian Restaurant,Bar,Bakery
6,Santa Cecília,Pizza Place,Pet Store,Italian Restaurant,Dessert Shop,Japanese Restaurant
7,Se,Cosmetics Shop,Japanese Restaurant,Grocery Store,Bakery,Bookstore
9,Carrao,Restaurant,Bar,Gym / Fitness Center,Ice Cream Shop,BBQ Joint
10,Vila Formosa,Plaza,Bakery,Gym / Fitness Center,Spa,Cosmetics Shop


In [110]:
saopaulo_merged.loc[saopaulo_merged['Cluster Labels'] ==2 , saopaulo_merged.columns[[1] + list(range(5, saopaulo_merged.shape[1]))]]

,Boroughs,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
48,Anhanguera,Tennis Court,Soccer Field,Yoga Studio,Food,Farmers Market


In [111]:
saopaulo_merged.loc[saopaulo_merged['Cluster Labels'] == 3, saopaulo_merged.columns[[1] + list(range(5, saopaulo_merged.shape[1]))]]

,Boroughs,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
11,Cidade Tiradentes,Athletics & Sports,Yoga Studio,Falafel Restaurant,Farmers Market,Fast Food Restaurant


In [112]:
saopaulo_merged.loc[saopaulo_merged['Cluster Labels'] == 4, saopaulo_merged.columns[[1] + list(range(5, saopaulo_merged.shape[1]))]]

,Boroughs,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
47,Perus,Ice Cream Shop,Yoga Studio,Food & Drink Shop,Farmers Market,Fast Food Restaurant


In [113]:
saopaulo_merged.loc[saopaulo_merged['Cluster Labels'] == 5, saopaulo_merged.columns[[1] + list(range(5, saopaulo_merged.shape[1]))]]

,Boroughs,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
51,Sao Domingos,Brazilian Restaurant,Food & Drink Shop,Farmers Market,Fast Food Restaurant,Festival


In [114]:
saopaulo_merged.loc[saopaulo_merged['Cluster Labels'] == 6, saopaulo_merged.columns[[1] + list(range(5, saopaulo_merged.shape[1]))]]

,Boroughs,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
13,Ponte Rasa,Gym / Fitness Center,Road,Bakery,Seafood Restaurant,Supermarket
17,Vila Curuca,Pizza Place,Food,Drugstore,Grocery Store,Farmers Market
19,Cidade Lider,Supermarket,Gastropub,Gym / Fitness Center,Market,Paper / Office Supplies Store
21,Parque do Carmo,Pizza Place,Bakery,Burger Joint,Acai House,Food Stand
27,Tatuape,Restaurant,Pizza Place,Gym / Fitness Center,Bakery,Pet Store
28,Penha,Soccer Stadium,Gym / Fitness Center,Athletics & Sports,Pizza Place,Bus Station
29,Artur Alvim,Pizza Place,Soccer Stadium,Grocery Store,Burger Joint,Clothing Store
32,Sao Mateus,Market,Pizza Place,Bus Stop,Men's Store,Yoga Studio
35,Jardim Helena,Gym / Fitness Center,Pizza Place,Lottery Retailer,Fast Food Restaurant,Restaurant
37,Sapopemba,Pizza Place,Farmers Market,Food Truck,Video Game Store,Sandwich Place


In [115]:
saopaulo_merged.loc[saopaulo_merged['Cluster Labels'] == 7, saopaulo_merged.columns[[1] + list(range(5, saopaulo_merged.shape[1]))]]

,Boroughs,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
57,Vila Medeiros,Bakery,Food & Drink Shop,Gym / Fitness Center,Diner,Food


In [116]:
saopaulo_merged.loc[saopaulo_merged['Cluster Labels'] == 8, saopaulo_merged.columns[[1] + list(range(5, saopaulo_merged.shape[1]))]]

,Boroughs,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
8,Aricanduva,Bakery,Pharmacy,Pastelaria,Café,Gym
15,Lajeado,Sandwich Place,Bakery,Gymnastics Gym,Gym,Ice Cream Shop
33,Sao Rafael,Bakery,Brazilian Restaurant,Pastelaria,Clothing Store,Farmers Market
65,Jaguara,Gym,Bakery,Park,Farmers Market,Seafood Restaurant
78,Grajau,Ice Cream Shop,Bakery,Food Truck,Soccer Stadium,Yoga Studio


In [117]:
saopaulo_merged.loc[saopaulo_merged['Cluster Labels'] == 9, saopaulo_merged.columns[[1] + list(range(5, saopaulo_merged.shape[1]))]]

,Boroughs,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
12,Ermelino Matarazzo,Restaurant,Soccer Field,Yoga Studio,Flower Shop,Farm
